In [35]:
import pandas as pd
import datetime

# Khởi tạo dataframe rỗng để lưu trữ tất cả dữ liệu
all_data = pd.DataFrame()

# Bắt đầu từ trang 1
page = 1

while True:
    # Tạo URL với số trang hiện tại
    start_date = '2025-01-02'
    end_date = datetime.date.today().strftime('%Y-%m-%d')
    start_date_formatted = datetime.datetime.strptime(start_date, '%Y-%m-%d').date().strftime('%d/%m/%Y')
    end_date_formatted = datetime.datetime.strptime(end_date, '%Y-%m-%d').date().strftime('%d/%m/%Y')

    url = f'https://dulieu.nguoiquansat.vn/History/PhanLoaiNDTHistory?page={page}&fromDate={start_date_formatted}&toDate={end_date_formatted}&exId=&code=VN-Index&idNganh=&_=1769924091168'
    
    try:
        # Đọc dữ liệu từ URL
        table = pd.read_html(url, encoding='utf-8')
        df = pd.DataFrame(table[0])
        
        # Kiểm tra nếu dataframe rỗng thì thoát vòng lặp
        if df.empty:
            print(f"Không có dữ liệu ở trang {page}. Dừng vòng lặp.")
            break
        
        # Ghép dataframe mới vào dataframe tổng
        all_data = pd.concat([all_data, df], ignore_index=True)
        # print(f"Đã lấy dữ liệu từ trang {page}. Tổng số dòng: {len(all_data)}")
        
        # Tăng số trang lên
        page += 1
        
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ trang {page}: {e}")
        break

# print(f"Hoàn thành! Tổng số dòng dữ liệu: {len(all_data)}")
# print(all_data.head())

# Nếu bạn muốn thêm độ trễ giữa các lần request để tránh bị chặn:

# import time

# # Khởi tạo dataframe rỗng
# all_data = pd.DataFrame()
# page = 1

# while True:
#     url = f'https://dulieu.nguoiquansat.vn/History/PhanLoaiNDTHistory?page={page}&fromDate={start_date_formatted}&toDate={end_date_formatted}&exId=&code=VN-Index&idNganh=&_=1769924091168'
    
#     try:
#         table = pd.read_html(url, encoding='utf-8')
#         df = pd.DataFrame(table[0])
        
#         if df.empty:
#             print(f"Không có dữ liệu ở trang {page}. Dừng vòng lặp.")
#             break
        
#         all_data = pd.concat([all_data, df], ignore_index=True)
#         print(f"Đã lấy dữ liệu từ trang {page}. Tổng số dòng: {len(all_data)}")
        
#         page += 1
        
#         # Đợi 1 giây trước khi request tiếp theo
#         time.sleep(1)
        
#     except Exception as e:
#         print(f"Lỗi khi lấy dữ liệu từ trang {page}: {e}")
#         break

# print(f"Hoàn thành! Tổng số dòng dữ liệu: {len(all_data)}")

Không có dữ liệu ở trang 19. Dừng vòng lặp.


In [36]:
all_data.columns

MultiIndex([(                    'STT',            'STT'),
            (                   'Ngày',           'Ngày'),
            (                  'Mã CK',          'Mã CK'),
            (           'Giá đóng cửa',   'Giá đóng cửa'),
            (          'Tự doanh ròng', 'GTGD khớp lệnh'),
            (          'Tự doanh ròng',      'Tổng GTGD'),
            ('Cá nhân trong nước ròng', 'GTGD khớp lệnh'),
            ('Cá nhân trong nước ròng',      'Tổng GTGD'),
            ('Tổ chức trong nước ròng', 'GTGD khớp lệnh'),
            ('Tổ chức trong nước ròng',      'Tổng GTGD'),
            ('Cá nhân nước ngoài ròng', 'GTGD khớp lệnh'),
            ('Cá nhân nước ngoài ròng',      'Tổng GTGD'),
            ('Tổ chức nước ngoài ròng', 'GTGD khớp lệnh'),
            ('Tổ chức nước ngoài ròng',      'Tổng GTGD')],
           )

In [37]:
# Xóa các cột có level 1 là 'GTGD khớp lệnh'
cols_to_drop = [col for col in all_data.columns if col[1] == 'GTGD khớp lệnh']
all_data = all_data.drop(columns=cols_to_drop)

# Kiểm tra kết quả
print("Các cột còn lại sau khi xóa 'GTGD khớp lệnh':")
for col in all_data.columns:
    print(f"  Level 0: '{col[0]}' | Level 1: '{col[1]}'")


Các cột còn lại sau khi xóa 'GTGD khớp lệnh':
  Level 0: 'STT' | Level 1: 'STT'
  Level 0: 'Ngày' | Level 1: 'Ngày'
  Level 0: 'Mã CK' | Level 1: 'Mã CK'
  Level 0: 'Giá đóng cửa' | Level 1: 'Giá đóng cửa'
  Level 0: 'Tự doanh ròng' | Level 1: 'Tổng GTGD'
  Level 0: 'Cá nhân trong nước ròng' | Level 1: 'Tổng GTGD'
  Level 0: 'Tổ chức trong nước ròng' | Level 1: 'Tổng GTGD'
  Level 0: 'Cá nhân nước ngoài ròng' | Level 1: 'Tổng GTGD'
  Level 0: 'Tổ chức nước ngoài ròng' | Level 1: 'Tổng GTGD'


In [38]:
# Xử lý multi-level index columns
all_data.columns = [
    col[0] if col[0] == col[1] else f"{col[0]} {col[1]}"
    for col in all_data.columns
]

# Kiểm tra kết quả
print("Các cột sau khi xử lý:")
print(all_data.columns.tolist())
print(f"\nSố cột: {len(all_data.columns)}")
print("\n5 dòng đầu tiên:")
print(all_data.head())


Các cột sau khi xử lý:
['STT', 'Ngày', 'Mã CK', 'Giá đóng cửa', 'Tự doanh ròng Tổng GTGD', 'Cá nhân trong nước ròng Tổng GTGD', 'Tổ chức trong nước ròng Tổng GTGD', 'Cá nhân nước ngoài ròng Tổng GTGD', 'Tổ chức nước ngoài ròng Tổng GTGD']

Số cột: 9

5 dòng đầu tiên:
   STT        Ngày     Mã CK  Giá đóng cửa  Tự doanh ròng Tổng GTGD  \
0    1  30/01/2026  VN-Index       1829.04                   187.29   
1    2  29/01/2026  VN-Index       1814.98                    52.68   
2    3  28/01/2026  VN-Index       1802.91                   118.93   
3    4  27/01/2026  VN-Index       1830.50                   639.08   
4    5  26/01/2026  VN-Index       1843.72                   350.28   

   Cá nhân trong nước ròng Tổng GTGD  Tổ chức trong nước ròng Tổng GTGD  \
0                            -707.95                            -164.40   
1                            -277.17                             681.63   
2                             437.54                            1230.81   
3    

In [40]:
# Bỏ cột 'STT'
if 'STT' in all_data.columns:
    all_data = all_data.drop(columns=['STT'])

# Xóa đoạn chữ 'Tổng GTGD' khỏi tên các cột (xử lý cả có/không có khoảng trắng)
all_data.columns = all_data.columns.str.replace('Tổng GTGD', '', regex=False)

# Xóa khoảng trắng thừa ở đầu/cuối tên cột
all_data.columns = all_data.columns.str.strip()

# Kiểm tra kết quả
print("Các cột sau khi xử lý:")
print(all_data.columns.tolist())
print(f"\nSố cột: {len(all_data.columns)}")
print("\n5 dòng đầu tiên:")
print(all_data.head())


Các cột sau khi xử lý:
['Ngày', 'Mã CK', 'Giá đóng cửa', 'Tự doanh ròng', 'Cá nhân trong nước ròng', 'Tổ chức trong nước ròng', 'Cá nhân nước ngoài ròng', 'Tổ chức nước ngoài ròng']

Số cột: 8

5 dòng đầu tiên:
         Ngày     Mã CK  Giá đóng cửa  Tự doanh ròng  Cá nhân trong nước ròng  \
0  30/01/2026  VN-Index       1829.04         187.29                  -707.95   
1  29/01/2026  VN-Index       1814.98          52.68                  -277.17   
2  28/01/2026  VN-Index       1802.91         118.93                   437.54   
3  27/01/2026  VN-Index       1830.50         639.08                  -296.64   
4  26/01/2026  VN-Index       1843.72         350.28                 -1161.94   

   Tổ chức trong nước ròng  Cá nhân nước ngoài ròng  Tổ chức nước ngoài ròng  
0                  -164.40                   257.85                   427.21  
1                   681.63                   190.01                  -647.16  
2                  1230.81                  2410.95              

In [41]:
# Chuyển đổi cột 'Ngày' sang định dạng datetime
all_data['Ngày'] = pd.to_datetime(all_data['Ngày'], format='%d/%m/%Y')

# Sắp xếp dataframe theo cột 'Ngày' tăng dần
all_data = all_data.sort_values('Ngày')

# Reset index sau khi sắp xếp
all_data = all_data.reset_index(drop=True)

# Kiểm tra kết quả
print("Các cột sau khi xử lý:")
print(all_data.columns.tolist())
print(f"\nSố dòng: {len(all_data)}")
print("\n5 dòng đầu tiên:")
print(all_data.head())
print("\n5 dòng cuối cùng:")
print(all_data.tail())


Các cột sau khi xử lý:
['Ngày', 'Mã CK', 'Giá đóng cửa', 'Tự doanh ròng', 'Cá nhân trong nước ròng', 'Tổ chức trong nước ròng', 'Cá nhân nước ngoài ròng', 'Tổ chức nước ngoài ròng']

Số dòng: 266

5 dòng đầu tiên:
        Ngày     Mã CK  Giá đóng cửa  Tự doanh ròng  Cá nhân trong nước ròng  \
0 2025-01-02  VN-Index       1269.71        -765.77                  1248.68   
1 2025-01-03  VN-Index       1254.59        -991.42                  1555.37   
2 2025-01-06  VN-Index       1246.35         289.52                  -705.95   
3 2025-01-07  VN-Index       1246.95        -163.47                     7.88   
4 2025-01-08  VN-Index       1251.02         455.90                  -516.48   

   Tổ chức trong nước ròng  Cá nhân nước ngoài ròng  Tổ chức nước ngoài ròng  
0                  -386.22                    17.85                  -114.53  
1                   171.51                    14.01                  -749.47  
2                   303.30                     3.54                 

In [42]:
def investor_type(symbol='VN-Index', start_date=None, end_date=None):
    """
    Hàm lấy dữ liệu phân loại nhà đầu tư từ nguoiquansat.vn
    
    Parameters:
    - symbol (str): Mã chứng khoán, mặc định là 'VN-Index'
    - start_date (str): Ngày bắt đầu (định dạng yyyy-mm-dd)
    - end_date (str): Ngày kết thúc (định dạng yyyy-mm-dd), mặc định là ngày hiện tại
    
    Returns:
    - DataFrame: DataFrame chứa dữ liệu phân loại nhà đầu tư đã được xử lý
    """
    import pandas as pd
    import datetime
    
    # Xử lý ngày kết thúc
    if end_date is None:
        end_date = datetime.date.today().strftime('%Y-%m-%d')
    
    # Định dạng lại ngày cho URL
    start_date_formatted = datetime.datetime.strptime(start_date, '%Y-%m-%d').date().strftime('%d/%m/%Y')
    end_date_formatted = datetime.datetime.strptime(end_date, '%Y-%m-%d').date().strftime('%d/%m/%Y')
    
    # Khởi tạo dataframe rỗng
    all_data = pd.DataFrame()
    page = 1
    
    while True:
        # Tạo URL với số trang hiện tại
        url = f'https://dulieu.nguoiquansat.vn/History/PhanLoaiNDTHistory?page={page}&fromDate={start_date_formatted}&toDate={end_date_formatted}&exId=&code={symbol}&idNganh=&_=1769924091168'
        
        try:
            # Đọc dữ liệu từ URL
            table = pd.read_html(url, encoding='utf-8')
            df = pd.DataFrame(table[0])
            
            # Kiểm tra nếu dataframe rỗng thì thoát vòng lặp
            if df.empty:
                break
            
            # Ghép dataframe mới vào dataframe tổng
            all_data = pd.concat([all_data, df], ignore_index=True)
            
            # Tăng số trang lên
            page += 1
            
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu từ trang {page}: {e}")
            break
    
    # Xử lý dataframe
    # 1. Xóa các cột có level 1 là 'GTGD khớp lệnh'
    if isinstance(all_data.columns, pd.MultiIndex):
        cols_to_drop = [col for col in all_data.columns if col[1] == 'GTGD khớp lệnh']
        all_data = all_data.drop(columns=cols_to_drop)
        
        # 2. Xử lý multi-level index columns
        all_data.columns = [
            col[0] if col[0] == col[1] else f"{col[0]} {col[1]}"
            for col in all_data.columns
        ]
    
    # 3. Bỏ cột 'STT'
    if 'STT' in all_data.columns:
        all_data = all_data.drop(columns=['STT'])
    
    # 4. Xóa đoạn chữ 'Tổng GTGD' khỏi tên các cột
    all_data.columns = all_data.columns.str.replace('Tổng GTGD', '', regex=False)
    all_data.columns = all_data.columns.str.strip()
    
    # 5. Chuyển đổi cột 'Ngày' sang định dạng datetime và sắp xếp
    all_data['Ngày'] = pd.to_datetime(all_data['Ngày'], format='%d/%m/%Y')
    all_data = all_data.sort_values('Ngày')
    all_data = all_data.reset_index(drop=True)
    
    return all_data


In [ ]:
# Sử dụng với tham số mặc định (VN-Index, ngày hiện tại)
# df = investor_type(start_date='2025-01-02')
# print(df.head())

# Sử dụng với tham số tùy chỉnh
# df = investor_type(symbol='HPG', start_date='2024-01-01', end_date='2024-12-31')
# print(df.head())


        Ngày Mã CK  Giá đóng cửa  Tự doanh ròng  Cá nhân trong nước ròng  \
0 2024-09-17   HPG         25.25          -2.97                    46.09   
1 2024-09-18   HPG         25.10          -6.25                    85.82   
2 2024-09-19   HPG         25.25           5.16                    71.25   
3 2024-09-20   HPG         25.65        -144.72                   222.25   
4 2024-09-23   HPG         25.70         -85.18                    67.15   

   Tổ chức trong nước ròng  Cá nhân nước ngoài ròng  Tổ chức nước ngoài ròng  
0                   -34.39                     0.51                    -9.23  
1                     7.63                    -6.53                   -80.66  
2                    -1.52                    -0.15                   -74.74  
3                    -8.27                     1.12                   -70.38  
4                    17.90                     5.95                    -5.81  
